<a href="https://colab.research.google.com/github/NULabTMN/ps4-AnantShanbhag93/blob/master/CrossLanguageEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cross-Language Word Embeddings

We have mentioned, and will discuss in more detail this week, how we can reduce the dimensionality of word representations from their original vectors space to an embedding space on the order of a few hundred dimensions. Different modeling choices for word embeddings may be ultimately evaluated by the effectiveness of classifiers, parsers, and other inference models that use those embeddings.

In this assignment, however, we will consider another common method of evaluating word embeddings: by judging the usefulness of pairwise distances between words in the embedding space.

Follow along with the examples in this notebook, and implement the sections of code flagged with **TODO**.

In [0]:
import gensim
import numpy as np
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

We'll start by downloading a plain-text version of the Shakespeare plays we used for the first assignment.

In [3]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/shakespeare_plays.txt
lines = [s.split() for s in open('shakespeare_plays.txt')]

--2020-04-20 14:05:41--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/shakespeare_plays.txt
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4746840 (4.5M) [text/plain]
Saving to: ‘shakespeare_plays.txt’

shakespeare_plays.t 100%[===================>]   4.53M  3.13MB/s    in 1.4s    

2020-04-20 14:05:43 (3.13 MB/s) - ‘shakespeare_plays.txt’ saved [4746840/4746840]



Then, we'll estimate a simple word2vec model on the Shakespeare texts.

In [0]:
model = Word2Vec(lines)

Even with such a small training set size, you can perform some standard analogy tasks.

In [5]:
model.wv.most_similar(positive=['king','woman'], negative=['man'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('queen', 0.8013672232627869),
 ('prince', 0.7768323421478271),
 ('warwick', 0.7431796193122864),
 ('york', 0.7204311490058899),
 ('clarence', 0.7182973027229309),
 ('margaret', 0.700549840927124),
 ('duke', 0.69575035572052),
 ('princess', 0.692003607749939),
 ('gloucester', 0.684795618057251),
 ('cardinal', 0.6820204257965088)]

For the rest of this assignment, we will focus on finding words with similar embeddings, both within and across languages. For example, what words are similar to the name of the title character of *Othello*?

In [6]:
model.wv.most_similar(positive=['othello'])
#model.wv.most_similar(positive=['brutus'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('desdemona', 0.9635088443756104),
 ('cleopatra', 0.9403950572013855),
 ('iago', 0.9370341300964355),
 ('valentine', 0.9258862733840942),
 ('cassio', 0.9208383560180664),
 ('emilia', 0.9184393882751465),
 ('proteus', 0.9165088534355164),
 ('ham', 0.9153567552566528),
 ('ford', 0.9076030254364014),
 ('glou', 0.9026821851730347)]

This search uses cosine similarity. In the default API, you should see the same similarity between the words `othello` and `desdemona` as in the search results above.

In [7]:
model.wv.similarity('othello', 'desdemona')
model.wv.similarity('queen', 'king')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.72393024

**TODO**: Your **first task** therefore is to implement your own cosine similarity function so that you can reuse it outside of the context of the gensim model object.

In [8]:
## TODO: Implement cosim
def cosim(v1, v2):
  ## return cosine similarity between v1 and v2
  num = np.dot(v1,v2)
  v1_sq = np.square(v1)
  v2_sq = np.square(v2)
  v1_sq_sum = np.sum(v1_sq)
  v2_sq_sum = np.sum(v2_sq)
  sim = num / (np.sqrt(v1_sq_sum) * np.sqrt(v2_sq_sum))
  return sim

## This should give a result similar to model.wv.similarity:
cosim(model.wv['othello'], model.wv['desdemona'])
cosim(model.wv['king'], model.wv['queen'])

0.72393024

## Evaluation

We could collect a lot of human judgments about how similar pairs of words, or pairs of Shakespearean characters, are. Then we could compare different word-embedding models by their ability to replicate these human judgments.

If we extend our ambition to multiple languages, however, we can use a word translation task to evaluate word embeddings.

We will use a subset of [Facebook AI's FastText cross-language embeddings](https://fasttext.cc/docs/en/aligned-vectors.html) for several languages. Your task will be to compare English both to French, and to *one more language* from the following set: Arabic, German, Portuguese, Russian, Spanish, Vietnamese, and Chinese.

In [9]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.en.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.fr.vec

# TODO: uncomment at least one of these
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.ar.vec
# wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.de.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.pt.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.ru.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.es.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.vi.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.zh.vec

--2020-04-20 14:05:54--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.en.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67681172 (65M)
Saving to: ‘30k.en.vec’

30k.en.vec          100%[===================>]  64.54M  15.6MB/s    in 5.4s    

2020-04-20 14:06:00 (12.0 MB/s) - ‘30k.en.vec’ saved [67681172/67681172]

--2020-04-20 14:06:01--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.fr.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67802327 (65M)
Saving to: ‘30k.fr.vec’

30k.fr.vec          100%[===================>]  64.66M  15.9MB/s    in 5.3s    

2020-04-20 14:06:06 (12.1 MB/s) - ‘30k.fr.vec’ saved [67802327/67802327]

--2020-04-20 14:06:07--  http://www.ccs.neu.

We'll start by loading the word vectors from their textual file format to a dictionary mapping words to numpy arrays.

In [0]:
def vecref(s):
  (word, srec) = s.split(' ', 1)
  return (word, np.fromstring(srec, sep=' '))

def ftvectors(fname):
  return { k:v for (k, v) in [vecref(s) for s in open(fname)] if len(v) > 1} 

envec = ftvectors('30k.en.vec')
frvec = ftvectors('30k.fr.vec')

# TODO: load vectors for one more language, such as zhvec
# arvec = ftvectors('30k.ar.vec')
# devec = ftvectors('30k.de.vec')
# ptvec = ftvectors('30k.pt.vec')
# ruvec = ftvectors('30k.ru.vec')
esvec = ftvectors('30k.es.vec')
# vivec = ftvectors('30k.vi.vec')
# zhvec = ftvectors('30k.zh.vec')

**TODO**: Your next task is to write a simple function that takes a vector and a dictionary of vectors and finds the most similar item in the dictionary. For this assignment, a linear scan through the dictionary using your `cosim` function from above is acceptible.

In [11]:
## TODO: implement this search function
def mostSimilar(vec, vecDict):
  ## Use your cosim function from above
  max = 0;
  mostSimilar = ''
  for word,vec2 in vecDict.items():
    sim = cosim(vec, vec2)
    if sim>max:
      max = sim
      mostSimilar = word
  similarity = max
  return (mostSimilar, similarity)

## some example searches
[mostSimilar(envec[e], frvec) for e in ['computer', 'germany', 'matrix', 'physics', 'yeast']]

[('informatique', 0.5023827767603765),
 ('allemagne', 0.5937184138759639),
 ('matrice', 0.5088361302065516),
 ('physique', 0.4555543434796393),
 ('fermentation', 0.3504105196166514)]

Some matches make more sense than others. Note that `computer` most closely matches `informatique`, the French term for *computer science*. If you looked further down the list, you would see `ordinateur`, the term for *computer*. This is one weakness of a focus only on embeddings for word *types* independent of context.

To evalute cross-language embeddings more broadly, we'll look at a dataset of links between Wikipedia articles.

In [12]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/links.tab
links = [s.split() for s in open('links.tab')]

--2020-04-20 14:06:22--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/links.tab
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1408915 (1.3M)
Saving to: ‘links.tab’

links.tab           100%[===================>]   1.34M  1.25MB/s    in 1.1s    

2020-04-20 14:06:23 (1.25 MB/s) - ‘links.tab’ saved [1408915/1408915]



This `links` variable consists of triples of `(English term, language, term in that language)`. For example, here is the link between English `academy` and French `académie`:

In [13]:
links[302]

['academy', 'fr', 'académie']

In [14]:
print(links[:100])

[['a', 'ar', 'a'], ['a', 'de', 'a'], ['a', 'es', 'a'], ['a', 'ja', 'a'], ['a', 'pt', 'a'], ['a', 'vi', 'a'], ['a', 'zh', 'a'], ['aachen', 'ar', 'آخن'], ['aachen', 'de', 'aachen'], ['aachen', 'es', 'aquisgrán'], ['aachen', 'ja', 'アーヘン'], ['aachen', 'pt', 'aachen'], ['aachen', 'ru', 'ахен'], ['aachen', 'vi', 'aachen'], ['aachen', 'zh', '亚琛'], ['aalborg', 'ar', 'آلبورغ'], ['aalborg', 'de', 'aalborg'], ['aalborg', 'es', 'aalborg'], ['aalborg', 'fr', 'aalborg'], ['aalborg', 'ja', 'オールボー'], ['aalborg', 'pt', 'aalborg'], ['aalborg', 'ru', 'ольборг'], ['aalborg', 'vi', 'aalborg'], ['aalborg', 'zh', '奥尔堡'], ['aarhus', 'ar', 'آرهوس'], ['aarhus', 'de', 'aarhus'], ['aarhus', 'es', 'aarhus'], ['aarhus', 'fr', 'aarhus'], ['aarhus', 'ja', 'オーフス'], ['aarhus', 'pt', 'aarhus'], ['aarhus', 'ru', 'орхус'], ['aarhus', 'vi', 'aarhus'], ['aarhus', 'zh', '奥胡斯'], ['aaron', 'ar', 'هارون'], ['aaron', 'es', 'aarón'], ['aaron', 'ja', 'アロン'], ['aaron', 'pt', 'aarão'], ['aaron', 'ru', 'аарон'], ['aaron', 'vi', 'aaro

**TODO**: Evaluate the English and French embeddings by computing the proportion of English Wikipedia articles whose corresponding French article is also the closest word in embedding space. Skip English articles not covered by the word embedding dictionary. Since many articles, e.g., about named entities have the same title in English and French, compute the baseline accuracy achieved by simply echoing the English title as if it were French. Remember to iterate only over English Wikipedia articles, not the entire embedding dictionary.

In [0]:
## TODO: Compute English-French Wikipedia retrieval accuracy.
def calc_accuracy(links, lang_dict, lang):
  accuracy = 0
  baselineAccuracy = 0
  count = 0
  for (eng_word, t_lang, lang_word) in links:
    if t_lang == lang and eng_word in envec:
      count +=1
      eng_vec = envec[eng_word]
      most_similar, similarity = mostSimilar(eng_vec, lang_dict)
      if most_similar==lang_word:
        accuracy += 1
      if eng_word==lang_word:
        baselineAccuracy +=1
  print("acc ",accuracy)
  print("bacc ",baselineAccuracy)
  print("count ",count)
  accuracy = accuracy / count
  baselineAccuracy = baselineAccuracy / count

  return (accuracy, baselineAccuracy)

In [16]:
accuracy, baselineAccuracy = calc_accuracy(links, frvec, "fr")
print(accuracy, baselineAccuracy)

acc  5289
bacc  6654
count  9869
0.5359205593271862 0.6742324450298915


**TODO**: Compute accuracy and baseline (identity function) acccuracy for Englsih and another language besides French. Although the baseline will be lower for languages not written in the Roman alphabet (i.e., Arabic or Chinese), there are still many articles in those languages with headwords written in Roman characters.

In [17]:
accuracy, baselineAccuracy = calc_accuracy(links, esvec, "es")
print(accuracy, baselineAccuracy)

acc  4355
bacc  4147
count  8016
0.5432884231536926 0.5173403193612774


Further evaluation, if you are interested, could involve looking at the $k$ nearest neighbors of each English term to compute "recall at 10" or "mean reciprocal rank at 10".

In [0]:
### The faster version is below

In [0]:
def cosim_fast(v1, v2, v1_sqrt_sum_sq, v2_sqrt_sum_sq):
  ## return cosine similarity between v1 and v2
  num = np.dot(v1,v2)
  sim = num / (v1_sqrt_sum_sq * v2_sqrt_sum_sq)
  return sim

In [0]:
def create_root_sq_sum_dict(dict_lang):
  root_sq_sum_dict = dict()
  for word, vec in dict_lang.items():
    vec_sq = np.square(vec)
    vec_sq_sum = np.sum(vec_sq)
    vec_root_sq_sum = np.sqrt(vec_sq_sum)
    root_sq_sum_dict[word] = vec_root_sq_sum
  return root_sq_sum_dict

In [0]:
## TODO: implement this search function
def mostSimilar_fast(vec, vec_sqrt, vecDict, map_vec2_sqrt):
  ## Use your cosim function from above
  max = 0;
  mostSimilar = ''
  for word,vec2 in vecDict.items():
    sim = cosim_fast(vec, vec2, vec_sqrt, map_vec2_sqrt[word])
    if sim>max:
      max = sim
      mostSimilar = word
  similarity = max
  return (mostSimilar, similarity)

In [0]:
## TODO: Compute English-French Wikipedia retrieval accuracy.
def calc_accuracy_fast(links, eng_dict, lang_dict, sq_eng_dict, sq_lang_dict, lang):
  accuracy = 0
  baselineAccuracy = 0
  count = 0
  for (eng_word, t_lang, lang_word) in links:
    if t_lang == lang and eng_word in envec:
      count +=1
      eng_vec = eng_dict[eng_word]
      most_similar, similarity = mostSimilar_fast(eng_vec, sq_eng_dict[eng_word], lang_dict, sq_lang_dict )
      if most_similar==lang_word:
        accuracy += 1
      if eng_word==lang_word:
        baselineAccuracy +=1
  print("acc ",accuracy)
  print("bacc ",baselineAccuracy)
  print("count ",count)
  accuracy = accuracy / count
  baselineAccuracy = baselineAccuracy / count
  return (accuracy, baselineAccuracy)

In [26]:
envec_root_sq_sum_dict = create_root_sq_sum_dict(envec)
frvec_root_sq_sum_dict = create_root_sq_sum_dict(frvec)
accuracy, baselineAccuracy = calc_accuracy_fast(links, envec, frvec, envec_root_sq_sum_dict,  frvec_root_sq_sum_dict, "fr")
print(accuracy, baselineAccuracy)

acc  5289
bacc  6654
count  9869
0.5359205593271862 0.6742324450298915


In [27]:
## TODO: Compute English-X Wikipedia retrieval accuracy.
envec_root_sq_sum_dict = create_root_sq_sum_dict(envec)
esvec_root_sq_sum_dict = create_root_sq_sum_dict(esvec)
accuracy, baselineAccuracy = calc_accuracy_fast(links, envec, esvec, envec_root_sq_sum_dict,  esvec_root_sq_sum_dict, "es")
print(accuracy, baselineAccuracy)

acc  4355
bacc  4147
count  8016
0.5432884231536926 0.5173403193612774
